# True match cleaning

In [1]:
#Import Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import jellyfish
%matplotlib inline

In [2]:
#Read in data and create comlumns
names = ["fsid", "Full1"] + ['Source{}'.format(x) for x in range(1,21)]
df = pd.read_csv('/Users/jperryman/Desktop/BYU/Python/api_scrape.csv', names=names)

In [3]:
df['Full1'].describe()

count     174790
unique    171406
top         Mary
freq          30
Name: Full1, dtype: object

In [4]:
#Clean noise
df.Source1 = df.Source1.str.replace('( in[\s\S]+)',"")

In [5]:
df.Source1 = df.Source1.str.extract('([A-Z][ \w]+)',expand = False)

In [6]:
#Clean sources of unneeded words and symbols
for x in ['Source{}'.format(x) for x in range(1,21)]:
    df[x] = df[x].str.replace('( in[\s\S]+)',"")
    df[x] = df[x].str.replace('Legacy NFS Source: ',"")
    df[x] = df[x].str.replace('[Ff]ind [Aa] [Gg]rave',"")
    df[x] = df[x].str.replace("\d","")
    df[x] = df[x].str.replace("( [a-z])\w+","")
    df[x] = df[x].str.replace("\.","")
    df[x] = df[x].str.replace('([Tt]he )?United States (Federal )?Census',"")
    df[x] = df[x].str.replace('( the )',"")
    df[x] = df[x].str.extract('([A-Z][ \w]+)',expand = False)

In [7]:
#Stack index
df = df.set_index(['fsid', 'Full1'])
df = df.stack().reset_index()

In [8]:
#Drop nickenames and suffixes
for x in ['Full1'.format(x)]:
    df[x] = df[x].str.replace('[\'\"]\w+[\'\"]',"")
    df[x] = df[x].str.replace('\"',"")
    df[x] = df[x].str.replace("\'","")
    df[x] = df[x].str.replace("( [a-z])\w+","")
    df[x] = df[x].str.replace('[\'\"]\w+[\'\"]',"")
    df[x] = df[x].str.replace('( [Jj]r)',"")
    df[x] = df[x].str.replace('( [Ss]r)',"")
    df[x] = df[x].str.replace('\.',"")

In [9]:
#Drop column level2
del df['level_2']

In [10]:
#Add column Full2
df.columns = ['fsid', 'Full1', 'Full2']

In [11]:
df['Full1'].describe()

count     1077774
unique     171170
top          Mary
freq          104
Name: Full1, dtype: object

In [12]:
#Drop nickenames and suffixes
for x in ['Full2'.format(x)]:
    df[x] = df[x].str.replace('\"',"")
    df[x] = df[x].str.replace('[\'\"]\w+[\'\"]',"")
    df[x] = df[x].str.replace("\'","")
    df[x] = df[x].str.replace("( [a-z])\w+","")
    df[x] = df[x].str.replace('( [Jj]r)',"")
    df[x] = df[x].str.replace('( [Ss]r)',"")
    df[x] = df[x].str.replace('\.',"")

In [13]:
#Drop Nan
df = df.fillna('')

In [14]:
#Find jarowinkler score for distances
df['score'] = df.apply(lambda row: jellyfish.jaro_distance(row['Full1'], row['Full2']), axis=1)

In [15]:
#Drop exact matches and outliers
df = df[df.score != 1]
df = df[df.score >= .75]

In [16]:
#strip columns
df['Full1'] = df['Full1'].str.strip()
df['Full2'] = df['Full2'].str.strip()

In [17]:
#Drop score and add first, middle, and last names
del df['score']
df['First1'] = df.Full1
df['Mid1'] = df.Full1
df['Last1'] = df.Full1
df['First2'] = df.Full2
df['Mid2'] = df.Full2
df['Last2'] = df.Full2

In [18]:
#organize columns
cols = list(df)
cols.insert(5, cols.pop(cols.index('Full2')))
cols

['fsid',
 'Full1',
 'First1',
 'Mid1',
 'Last1',
 'Full2',
 'First2',
 'Mid2',
 'Last2']

In [19]:
df = df.loc[:, cols]

In [20]:
#Extract first name
for x in ['First1'.format(x)]:
    df[x] = df[x].str.extract('(^\w+)', expand=False)

In [21]:
#Extract middle name
for x in ['Mid1'.format(x)]:
    df[x] = df[x].str.replace('^\w+',"")
    df[x] = df[x].str.replace('\w+$',"")

In [22]:
#Extract last name
for x in ['Last1'.format(x)]:
    df[x] = df[x].str.extract('(\w+$)', expand=False)

In [23]:
#Extract first name
for x in ['First2'.format(x)]:
    df[x] = df[x].str.extract('(^\w+)', expand=False)

In [24]:
#Extract middle name
for x in ['Mid2'.format(x)]:
    df[x] = df[x].str.replace('(\w+$)',"")
    df[x] = df[x].str.replace('(^\w+)',"")

In [25]:
#Extract last name
for x in ['Last2'.format(x)]:
    df[x] = df[x].str.extract('(\w+$)', expand=False)

In [26]:
#If first name = last name, drop last name
df.loc[df.First2 == df.Last2, "Last2"] = ""

In [27]:
#Drop Nan
df = df.fillna('')

In [28]:
#Create jaro scores
df['scoreFull'] = df.apply(lambda row: jellyfish.jaro_distance(row['Full1'], row['Full2']), axis=1)
df['scoreFirst'] = df.apply(lambda row: jellyfish.jaro_distance(row['First1'], row['First2']), axis=1)
df['scoreLast'] = df.apply(lambda row: jellyfish.jaro_distance(row['Last1'], row['Last2']), axis=1)

In [29]:
#Sum individual scores
df['score'] = sum((df['scoreFull'],df['scoreFirst']),df['scoreLast'])

In [30]:
#Drop scores that are outside of boundaries
df = df[df.score >= 1.50]

In [37]:
#Drop rows with no fsid
df = df[df.fsid != '']

In [31]:
#drop scores
del df['score']
del df['scoreFull']
del df['scoreFirst']
del df['scoreLast']

In [32]:
#Assign all rows Match=1
df['Match'] = 1

In [33]:
#Check work
df.head(50)

,fsid,Full1,First1,Mid1,Last1,Full2,First2,Mid2,Last2,Match
0,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Catharine Englehart,Catharine,,Englehart,1
1,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Katie Englehart,Katie,,Englehart,1
3,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip,E,Fuller,1
4,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip,E,Fuller,1
5,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip,E,Fuller,1
8,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip Fuller,Philip,,Fuller,1
11,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip,E,Fuller,1
12,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip,E,Fuller,1
13,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip,E,Fuller,1
15,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E,Philip,,E,1


In [38]:
df.head(28715)
#53008

,fsid,Full1,First1,Mid1,Last1,Full2,First2,Mid2,Last2,Match
0,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Catharine Englehart,Catharine,,Englehart,1
1,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Katie Englehart,Katie,,Englehart,1
3,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip,E,Fuller,1
4,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip,E,Fuller,1
5,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip,E,Fuller,1
8,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip Fuller,Philip,,Fuller,1
11,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip,E,Fuller,1
12,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip,E,Fuller,1
13,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip,E,Fuller,1
15,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E,Philip,,E,1


In [39]:
df['Full1'].describe()

count     591361
unique    146302
top         Mary
freq          74
Name: Full1, dtype: object

In [40]:
df.to_csv('/Users/jperryman/Desktop/BYU/Python/true_names.csv', index=False)